In [ ]:
import pandas as pd
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

def init_fee_config():
    return get_fee_config(
        min_fee = 100, 
        medium_fee = 500, 
        max_fee = 3000, 
        beta1 = 4200, 
        beta2 = 90000, 
        gamma1 = 700, 
        gamma2 = 15000
    )

def get_fee_config(min_fee, medium_fee, max_fee, beta1, beta2, gamma1, gamma2):
    config = {}
    config['baseFee'] = min_fee
    config['alpha1'] = medium_fee - config['baseFee']
    config['alpha2'] = max_fee - config['alpha1'] - config['baseFee']
    config['beta1'] = beta1
    config['beta2'] = beta2
    config['gamma1'] = gamma1
    config['gamma2'] = gamma2
    return config    

def sigmoid(volatility, alpha, beta, gamma):
    if (volatility > beta):
        volatility = volatility - beta
        if (volatility >= 6 * gamma):
            return alpha
        ex = math.exp(volatility / gamma) * gamma**4
        return (alpha * ex) // (gamma**4 + ex)
    else:
        volatility = beta - volatility
        if (volatility >= 6 * gamma):
            return 0
        ex = gamma**4 + math.exp(volatility / gamma) * gamma**4
        return (alpha * gamma**4) // ex;
        

def getFee(volatility, config):
    volatility = volatility // 15
    result = config['baseFee']
    result += sigmoid(volatility, config['alpha1'], config['beta1'], config['gamma1'])
    result +=  sigmoid(volatility, config['alpha2'], config['beta2'], config['gamma2'])
    return result

In [ ]:
def plot_with_ticks(df, resample_rate='1h'):
    df_resampled_1h = df.resample(resample_rate).mean()

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df_resampled_1h.index, y=df_resampled_1h['tick'], name='Tick'))
    fig.add_trace(go.Scatter(x=df_resampled_1h.index, y=df_resampled_1h['tickAverage'], name='Average tick, 24h'))
    fig.add_trace(go.Scatter(x=df_resampled_1h.index, y=df_resampled_1h['fee'], name='Fee'), secondary_y=True)

    fig.update_xaxes(title_text="Time")
    fig.update_yaxes(title_text="Tick value", secondary_y=False)
    fig.update_yaxes(title_text="Fee value", secondary_y=True)
    return fig
    
def plot_with_prices(df, decimals0, decimals1, inverse, resample_rate='1h'):
    df_resampled_1h = df.resample(resample_rate).mean()
    df_resampled_1h['price'] = (1.0001) ** df_resampled_1h['tick'] / (10**(decimals1 - decimals0))
    df_resampled_1h['priceAverage'] = (1.0001) ** df_resampled_1h['tickAverage'] / (10**(decimals1 - decimals0))
    if (inverse):
        df_resampled_1h['price'] = 1 / df_resampled_1h['price']
        df_resampled_1h['priceAverage'] = 1 / df_resampled_1h['priceAverage']
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df_resampled_1h.index, y=df_resampled_1h['price'], name='Price'))
    fig.add_trace(go.Scatter(x=df_resampled_1h.index, y=df_resampled_1h['priceAverage'], name='Average price, 24h'))
    fig.add_trace(go.Scatter(x=df_resampled_1h.index, y=df_resampled_1h['fee'], name='Fee'), secondary_y=True)

    fig.update_xaxes(title_text="Time")
    fig.update_yaxes(title_text="Price value", secondary_y=False, type="log")
    fig.update_yaxes(title_text="Fee value", secondary_y=True)
    return fig

In [ ]:
df = pd.read_json('input/timepoints.json')
df.set_index('timestamp', inplace=True)

In [ ]:
# Make plot with ticks, average ticks and corresponding fee values (default config)
plot_with_ticks(df)

In [ ]:
# Make plot with price, average prices and corresponding fee values (default config)
# Values are for ETH/USDC pair on Polygon. You should change them if you use other data
plot_with_prices(df, decimals0 = 6, decimals1 = 18, inverse = True)

In [ ]:
# Experimenting with fee configs

feeConfig = get_fee_config(
        min_fee = 100, 
        medium_fee = 3000, 
        max_fee = 15000, 
        beta1 = 360, 
        beta2 = 60000, 
        gamma1 = 59, 
        gamma2 = 8500
    )

feeConfig = get_fee_config(
        min_fee = 100, 
        medium_fee = 3000, 
        max_fee = 5000, 
        beta1 = 403, 
        beta2 = 8574, 
        gamma1 = 67, 
        gamma2 = 570
    )

default_new_token_config = get_fee_config(
        min_fee = 100, 
        medium_fee = 3000, 
        max_fee = 5000, 
        beta1 = 601, 
        beta2 = 8574, 
        gamma1 = 100, 
        gamma2 = 570
    )

df['fee_calculated'] = df.apply(lambda x: getFee(x['volatilityAverage'], default_new_token_config), axis=1)

In [ ]:
# plot new fee over price history
resampling_rate = '15min'
df_calculated = df.copy()
df_resampled = df_calculated.resample(resampling_rate).mean()
fig = plot_with_prices(df_calculated, decimals0 = 6, decimals1 = 18, inverse = True, resample_rate=resampling_rate)
fig.add_trace(go.Scatter(x=df_resampled.index, y=df_resampled['fee_calculated'], name='Fee (new)'), secondary_y=True)
fig.show()

In [ ]:
# plot difference between default and new fee

df_resampled_1h = df.resample('1h').mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_resampled_1h.index, y=df_resampled_1h['fee'], name='Fee'))
fig.add_trace(go.Scatter(x=df_resampled_1h.index, y=df_resampled_1h['fee_calculated'], name='Fee calculated'))

fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="Fee value")
fig.show()